# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [17]:
# import libraries
import re
import pickle
import pandas as pd 
from sqlalchemy import create_engine
import nltk 
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,  GridSearchCV 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster_Messages.db')
df = pd.read_sql_table('Disaster_Messages_Table',engine)
X = df['message'] 
y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        #print(clean_token)
        clean_tokens.append(clean_token)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = pipeline.predict(X_test)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [6]:
list(enumerate(y_test))

[(0, 'related'),
 (1, 'request'),
 (2, 'offer'),
 (3, 'aid_related'),
 (4, 'medical_help'),
 (5, 'medical_products'),
 (6, 'search_and_rescue'),
 (7, 'security'),
 (8, 'military'),
 (9, 'child_alone'),
 (10, 'water'),
 (11, 'food'),
 (12, 'shelter'),
 (13, 'clothing'),
 (14, 'money'),
 (15, 'missing_people'),
 (16, 'refugees'),
 (17, 'death'),
 (18, 'other_aid'),
 (19, 'infrastructure_related'),
 (20, 'transport'),
 (21, 'buildings'),
 (22, 'electricity'),
 (23, 'tools'),
 (24, 'hospitals'),
 (25, 'shops'),
 (26, 'aid_centers'),
 (27, 'other_infrastructure'),
 (28, 'weather_related'),
 (29, 'floods'),
 (30, 'storm'),
 (31, 'fire'),
 (32, 'earthquake'),
 (33, 'cold'),
 (34, 'other_weather'),
 (35, 'direct_report')]

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def report_performance(pipeline, X_test, y_test):
    y_pred = pipeline.predict(X_test)
    for i, column in enumerate(y_test):
        print(column)
        print(classification_report(y_test[column], y_pred[:, i]))
report_performance(pipeline, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.61      0.36      0.45      1459
          1       0.83      0.93      0.88      5045
          2       0.65      0.26      0.37        50

avg / total       0.78      0.80      0.78      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5433
          1       0.82      0.39      0.53      1121

avg / total       0.88      0.88      0.86      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.73      0.87      0.79      3835
          1       0.74      0.54      0.62      2719

avg / total       0.73      0.73      0.72      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fc3a865f510>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [10]:
parameters = {#'vect__min_df': [1, 3, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 50], 
              'clf__estimator__min_samples_split':[2, 4]}

#scorer = make_scorer(perform_metric)
cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 5, n_jobs=4)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, score=0.21391516631065, total= 3.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, score=0.21025328043942632, total= 3.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_i

[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed: 21.6min


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=True, score=0.21071101617332927, total= 2.7min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=True, score=0.21177906621910284, total= 2.8min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=True, score=0.2050656087885261, total= 2.8min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=False, score=0.20323466585291425, total= 2.8min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_

[Parallel(n_jobs=4)]: Done  22 out of  24 | elapsed: 44.8min remaining:  4.1min


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.22993591699725358, total= 5.3min
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.2239853524565151, total= 5.4min


[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed: 50.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [10, 50], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [12]:
report_performance(cv, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.75      0.27      0.40      1459
          1       0.82      0.97      0.89      5045
          2       0.72      0.26      0.38        50

avg / total       0.80      0.81      0.78      6554

request
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5433
          1       0.87      0.43      0.57      1121

avg / total       0.89      0.89      0.88      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.75      0.88      0.81      3835
          1       0.78      0.59      0.68      2719

avg / total       0.77      0.76      0.76      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
pipeline_new = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

parameters_new = {
    'clf__estimator__learning_rate': [0.1, 0.3],
    'clf__estimator__n_estimators': [100, 200]
}

cv_new = GridSearchCV(pipeline_new, param_grid=parameters_new, verbose=5, n_jobs=4)
cv_new.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100 
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100 
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100 
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=200 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100, score=0.21361000915471468, total=16.2min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=200 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100, score=0.20979554470552333, total=16.2min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=200 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100, score=0.2036924015868172, total=16.2min
[CV] clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=200, 

[Parallel(n_jobs=4)]: Done   8 out of  12 | elapsed: 51.2min remaining: 25.6min


[CV]  clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=100, score=0.2380225816295392, total=16.1min
[CV]  clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=200, score=0.2364967958498627, total=23.8min
[CV]  clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=200, score=0.23909063167531278, total=23.8min
[CV]  clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=200, score=0.24549893194995423, total=23.7min


[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 76.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'clf__estimator__learning_rate': [0.1, 0.3], 'clf__estimator__n_estimators': [100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [28]:
report_performance(cv_new, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.62      0.22      0.32      1459
          1       0.80      0.96      0.88      5045
          2       0.64      0.14      0.23        50

avg / total       0.76      0.79      0.75      6554

request
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5433
          1       0.80      0.53      0.64      1121

avg / total       0.89      0.90      0.89      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.76      0.88      0.81      3835
          1       0.78      0.60      0.68      2719

avg / total       0.77      0.76      0.76      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [30]:
with open('ml_model.pkl', 'wb') as f:
    pickle.dump(cv_new, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.